In [1]:
import torch
import sparseconvnet as scn
from next_sparseconvnet.networks.building_blocks import ResidualBlock_downsample, ResidualBlock_basic, ResidualBlock_upsample, ConvBNBlock

Para hacer la UResNet voy a construir una estructura del estilo

entrada:
* input
* submanifoldconvolution
* batchrenormReLU

izquierda:
* level1 : resnet_basic x 3 (por ejemplo 3, esto deberia ser un parametro creo)
* resnet_downsample
* level2 : resbnet basic x 3 ...

parte inferior:
* levelN : resnet basic x3

derecha:
* resnet_upsample
* levelN-1 : resnet basic x 3
* resnet upsample
* level N-2 : resnet basic x3

salid:
* outlayer
* linear (para obtener nclasses)

Hacer que el numero de veces que se aplica basic sea un parámetro (llamémosle basic_num) y la profundidad de niveles sea otro (N). Cada nivel para mi va a ser una altura de la U, contando tambien la de abajo como la última.

Voy a importar datos para ir viendo cómo quedan las cositas

In [2]:
import tables as tb
import pandas as pd
import re
import numpy as np

In [3]:
from next_sparseconvnet.utils.data_loaders import DataGen, collatefn, LabelType

In [4]:
from invisible_cities.io   import dst_io as dio

In [5]:
dataset_path = '/home/mperez/NEXT_SPARSECONVNET/next_sparseconvnet/test_files/MC_dataset.h5'

In [6]:
full_path = 'merged_beersheba_label_4mm_50scut.h5'

In [7]:
marija_path = '/home/mmkekic/MC_dataset/new_data/data/train_dataset_10_100.h5'

In [8]:
with tb.open_file(dataset_path) as h5in:
    print(h5in)

/home/mperez/NEXT_SPARSECONVNET/next_sparseconvnet/test_files/MC_dataset.h5 (File) ''
Last modif.: 'Thu Apr 22 16:28:51 2021'
Object Tree: 
/ (RootGroup) ''
/DATASET (Group) ''
/DATASET/BinsInfo (Table(1,), shuffle, zlib(4)) ''
/DATASET/EventsInfo (Table(6,), shuffle, zlib(4)) ''
/DATASET/Voxels (Table(652,), shuffle, zlib(4)) ''



In [9]:
with tb.open_file(full_path) as h5in:
    print(h5in)

merged_beersheba_label_4mm_50scut.h5 (File) ''
Last modif.: 'Tue May 24 10:58:54 2022'
Object Tree: 
/ (RootGroup) ''
/DATASET (Group) ''
/DATASET/BeershebaVoxels (Table(5796404,), shuffle, zlib(4)) ''
/DATASET/BinsInfo (Table(1,), shuffle, zlib(4)) ''
/DATASET/EventsInfo (Table(28767,), shuffle, zlib(4)) ''
/DATASET/IsauraInfo (Table(40719,), shuffle, zlib(4)) ''
/DATASET/MCHits (Table(7168681,), shuffle, zlib(4)) ''



In [10]:
dio.load_dst(dataset_path, 'DATASET', 'Voxels')

,xbin,ybin,zbin,energy,segclass,binclass,dataset_id
0,105,133,117,0.005435,1,1,0
1,105,145,111,0.008938,1,1,0
2,102,131,116,0.005346,1,1,0
3,106,133,113,0.003748,1,1,0
4,106,133,114,0.014020,1,1,0
...,...,...,...,...,...,...,...
647,100,150,114,0.004479,1,1,5
648,100,150,111,0.026472,0,1,5
649,100,149,111,0.002549,2,1,5
650,103,158,117,0.009361,1,1,5


In [11]:
dio.load_dst(dataset_path, 'DATASET', 'BinsInfo')

,min_x,max_x,nbins_x,min_y,max_y,nbins_y,min_z,max_z,nbins_z,Rmax
0,-200,200,201,-200,200,201,0,550,226,198


In [39]:
dio.load_dst(full_path, 'DATASET', 'BinsInfo')

,min_x,total_x,size_x,max_x,nbins_x,min_y,total_y,size_y,max_y,nbins_y,min_z,total_z,size_z,max_z,nbins_z,Rmax,loss_th,ener_th,sb_th
0,-220.0,440.0,5.0,220.0,89.0,-220.0,440.0,5.0,220.0,89.0,1.0,548.0,4.0,549.0,138.0,198.0,0.25,NaN,1.5


In [12]:
#vox = dio.load_dst(dataset_path, 'DATASET', 'Voxels')

In [13]:
#beersh_vox = dio.load_dst(full_path, 'DATASET', 'BeershebaVoxels')

In [14]:
binsInfo = dio.load_dst(full_path, 'DATASET', 'BinsInfo')

In [15]:
datagen_full = DataGen(full_path, LabelType.Segmentation, nevents = 2048, table = 'BeershebaVoxels')

In [16]:
datagen = DataGen(dataset_path, LabelType.Segmentation, nevents = None, table = 'Voxels')

In [29]:
batch = [datagen_full[i] for i in range(3)]
coord, ener, lab, ev = collatefn(batch)

El loader me hace batches de mis datos. Los parámetros importantes a priori son el batch_size y el drop_last.

batch_size dice cuántos eventos que cojo por batch: si tenemos 2000 eventos y batch size 1000, tendré 2 batches con 1000 eventos. Si el batch size fuera, digamos, 600, tendríamos 3 batches de 600 eventos (es decir, 1800 eventos en total) y un batch de 200. El drop_last decide si coger ese batch de distinto tamaño o no.

En la realidad probablemente tendré un número de eventos dificilmente divisible por un batch size lógico, por lo que siempre me quedará un batch residual. No sé muy bien por qué habríamos de desperdiciar un batch de datos poniendo drop_last True. Entiendo  que de alguna forma debe ser ''perjudicial'' para la red meterle un batch de distinto tamaño, si no no me tiene mucho sentido.

En contraposición, entiendo que un batch se pasa muchas veces (concretamente, nepoch es el número de veces que pasas todos tus batches creo), y como el loader tiene shuffle true, al final pues maomenos vamos a pasar todos esos datos y por eso no sería un problema...

In [18]:
loader = torch.utils.data.DataLoader(datagen,
                            batch_size = 3,
                            shuffle = True,
                            num_workers = 4,
                            collate_fn = collatefn,
                            drop_last = True, 
                            pin_memory = False)

In [19]:
loader_full = torch.utils.data.DataLoader(datagen_full,
                            batch_size = 1024,
                            shuffle = True,
                            num_workers = 4,
                            collate_fn = collatefn,
                            drop_last = True,
                            pin_memory = False)

In [30]:
type(lab)

torch.Tensor

In [31]:
ener.size()

torch.Size([534, 1])

In [32]:
dim = len(coord[0]) - 1

In [33]:
dim

3

In [34]:
coord.size()

torch.Size([534, 4])

In [35]:
ener.size()

torch.Size([534, 1])

In [36]:
coord

tensor([[ 62,  63, 102,   0],
        [ 62,  63, 104,   0],
        [ 63,  62, 101,   0],
        ...,
        [ 53,  64,  97,   2],
        [ 53,  64,  98,   2],
        [ 53,  65,  97,   2]])

In [45]:
inp = scn.InputLayer(dim, (149, 149, 149))
res_basic = ResidualBlock_basic(1, 2)
res_down = ResidualBlock_downsample(1, 3, 2) #inplanes, kernel, stride
out = scn.OutputLayer(dim)

x = inp((coord, ener))
print(x.spatial_size, x.features.shape)
x = res_basic(x)
print(x.spatial_size, x.features.shape)
x = res_down(x)
print(x.spatial_size, x.features.shape)

#Esta sería una salida de un downsample, la uso para la entrada de upsample
#El downsample tiene definido que los outplanes serán el doble de los inplanes
#Por eso, donde usé 1 outplane, ahora inPlanes (que es el número de features x así decirlo) es 2
# Al final al upsample se le mete el número de planes que tenía y por definición los divide por 2
#Así volvemos a tener 
inPlanes = x.features.shape[1]

res_up = ResidualBlock_upsample(inPlanes, 3, 2) #bien, parece que funciona bien
x = res_up(x)
print(x.spatial_size, x.features.shape)


x = out(x)
print(x.shape)

lin = torch.nn.Linear(1, 3)
x = lin(x)
print(x.shape)

tensor([149, 149, 149]) torch.Size([534, 1])
tensor([149, 149, 149]) torch.Size([534, 1])
tensor([74, 74, 74]) torch.Size([299, 2])
tensor([149, 149, 149]) torch.Size([534, 1])
torch.Size([534, 1])
torch.Size([534, 3])


In [40]:
def calculate_output_dimension(spatial_size, kernel_sizes, stride_sizes):
    """This function assures that kernel and stride sizes are suitable for our input size and returns the output size for the bottom layer"""
    out_dim = []
    for o in spatial_size:
        for i, k in enumerate(kernel_sizes[:-1]): 
            o = (o - k)/stride_sizes[i] + 1
            assert o == int(o), 'Shape mismatch: kernel size {} in level {} does not return a suitable size for the output'.format(k, i)
        out_dim.append(int(o))
    return out_dim

In [41]:
import pytest 

def test_calculate_output_dimension():
    spatial_size = (561, 561, 561)
    kernel_sizes = [9, 7, 5, 3, 3]
    stride_sizes = [4, 2, 2, 2]
    with pytest.raises(AssertionError):
        calculate_output_dimension(spatial_size, kernel_sizes, stride_sizes)
    
    spatial_size = (289, 417, 449)
    assert calculate_output_dimension(spatial_size, kernel_sizes, stride_sizes) == [7, 11, 12]    

In [42]:
test_calculate_output_dimension()

In [55]:
calculate_output_dimension((561, 561, 561), [9, 9, 7, 5, 3, 3, 3], [2, 2, 2, 2, 2, 2, 2])

[7, 7, 7]

In [59]:
calculate_output_dimension((149, 149, 149), [9, 9, 6, 4, 4], [2, 2, 2, 2]) #for the new data

[6, 6, 6]

In [61]:
class UNet(torch.nn.Module):
    '''
        This class implements a UNet structure, built with ResNet blocks. It takes a tuple of (coordinates, features) 
        and passes it through the UNet
        
        ...
        
        Attributes
        ----------
        spatial_size : tuple
            The spatial size of the input layer. Size of the tuple is also the dimension.
        inplanes : int
            Number of planes we want after the initial SubmanifoldConvolution, that is, to begin downsampling.
        kernel : list
            Size of the kernels applied in each layer or block. Its length also indicates the level depth of the UNet.
        stride : list
            Sizes of the strides applied in each downsample/upsample.
        conv_kernel : int
            Kernel for the first convolutional layer.
        basic_num : int
            Number of times a basic residual block is passed in each level.
        nclasses : int, optional
            Number of output classes for predictions. The default is 3.
        dim : int, optional
            Number of dimensions of the input. The default is 3.
        start_planes : int, optional
            Number of planes that enter the UNet. The default is 1.
            
        Methods
        -------
        forward(x)
            Passes the input through the UNet
        '''
    
    def __init__(self, spatial_size, init_conv_nplanes, init_conv_kernel, kernel_sizes, stride_sizes, basic_num, nclasses = 3, dim = 3, start_planes = 1):
        '''
            Parameters
            ----------
            spatial_size : tuple
                The spatial size of the input layer. Size of the tuple is also the dimension.
            init_conv_nplaness : int
                Number of planes we want after the initial SubmanifoldConvolution, that is, to begin downsampling.
            init_conv_kernel : int
                Kernel for the first convolutional layer.
            kernel_sizes : list
                Sizes of the kernels applied in each layer or block. Its length also indicates the level depth of the UNet.
                Last element corresponds just to the kernel of the basic block in the bottom of the UNet.
            stride_sizes : list
                Sizes of the strides applied in each downsample/upsample.
            basic_num : int
                Number of times a basic residual block is passed in each level.
            nclasses : int, optional
                Number of output classes for predictions. The default is 3.
            dim : int, optional
                Number of dimensions of the input. The default is 3.
            start_planes : int, optional
                Number of planes that enter the UNet. The default is 1.
        '''
        
        torch.nn.Module.__init__(self)
        
        self.basic_num   = basic_num
        self.level_depth = len(kernel_sizes)
        
        #Initial layers
        self.inp     = scn.InputLayer(dim, spatial_size)
        self.convBN  = ConvBNBlock(start_planes, init_conv_nplanes, init_conv_kernel)
        inplanes = init_conv_nplanes
        
        #Final layers
        self.output = scn.OutputLayer(dim)
        self.linear = torch.nn.Linear(inplanes, nclasses)
        
        #Branch layers
        self.downsample = torch.nn.ModuleList([])
        self.upsample   = torch.nn.ModuleList([])
        self.basic_down = torch.nn.ModuleList([torch.nn.ModuleList([]) for i in range(self.level_depth - 1)]) 
        self.bottom     = torch.nn.ModuleList([])
        self.basic_up   = torch.nn.ModuleList([torch.nn.ModuleList([]) for i in range(self.level_depth - 1)]) 
        
        for i in range(self.level_depth - 1):
            for j in range(basic_num):
                self.basic_down[i].append(ResidualBlock_basic(inplanes, kernel_sizes[i])) #basic blocks for downsample branch
                self.basic_up[i].append(ResidualBlock_basic(inplanes, kernel_sizes[i])) #basic blocks for upsample branch
            self.downsample.append(ResidualBlock_downsample(inplanes, kernel_sizes[i], stride_sizes[i])) #downsamples
            self.upsample.append(ResidualBlock_upsample(2 * inplanes, kernel_sizes[i], stride_sizes[i])) #upsamples, backwards
            
            inplanes = inplanes * 2 
        
        #Bottom layer
        for j in range(basic_num):
            self.bottom.append(ResidualBlock_basic(inplanes, kernel_sizes[-1])) #basic blocks for the lowest layer, kernel is the last element in the list
            
        self.add = scn.AddTable()
                
    def forward(self, x):
        '''
        Passes x through the UNet.

        Parameters
        ----------
        x : tuple
            It takes a tuple with (coord, features), where coord is a torch tensor with size [features_number, batch_size],
            and features is another torch tensor, with size [features_number, start_planes]

        Returns
        -------
        x : torch.Tensor
            A tensor with size [features_number, nclasses].

        '''
        x = self.inp(x)
        x = self.convBN(x)
        
        tmp_layers = []
        
        #downsample
        for i in range(self.level_depth - 1):
            for j in range(self.basic_num):
                x = self.basic_down[i][j](x)  
            tmp_layers.append(x) 
            x = self.downsample[i](x)

        #bottom
        for i in range(self.basic_num):
            x = self.bottom[i](x)
        
        #upsample
        for i in range(self.level_depth - 2, -1, -1):
            x = self.upsample[i](x)
            to_add = tmp_layers.pop()
            x = self.add([x, to_add])
            for j in range(self.basic_num):
                x = self.basic_up[i][j](x)
        
        x = self.output(x)
        x = self.linear(x)
        return x
        

In [179]:
red = UNet((149, 149, 149), 6, 3, [9, 9, 6, 4, 4], [2, 2, 2, 2], 3, nclasses=7) 


In [180]:
red.basic_up[0]

ModuleList(
  (0): ResidualBlock_basic(
    (bnr1): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
    (subconv1): SubmanifoldConvolution 6->6 C9
    (bnr2): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
    (subconv2): SubmanifoldConvolution 6->6 C9
    (add): AddTable()
  )
  (1): ResidualBlock_basic(
    (bnr1): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
    (subconv1): SubmanifoldConvolution 6->6 C9
    (bnr2): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
    (subconv2): SubmanifoldConvolution 6->6 C9
    (add): AddTable()
  )
  (2): ResidualBlock_basic(
    (bnr1): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
    (subconv1): SubmanifoldConvolution 6->6 C9
    (bnr2): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
    (subconv2): SubmanifoldConvolution 6->6 C9
    (add): AddTable()
  )
)

In [181]:
#spatial_size_ = {}
last_basic = []
last_out = []
#def get_spatial_size(name):
 #   def hook(model, input, output):
  #      spatial_size_[name] = [output.spatial_size, output.features.shape]
   # return hook

red.basic_up[0][-1].add.register_forward_hook(lambda model, input, output: last_basic.append([output.spatial_size, output.features.shape]))
red.output.register_forward_hook(lambda model, input, output: last_out.append(output.size()))

In [182]:
print(len(red.basic_up), len(red.basic_down), len(red.downsample), len(red.upsample), len(red.basic_up[0]))

4 4 4 4 3


In [183]:
out = red.forward((coord, ener, len(batch)))

In [184]:
print(last_basic, last_out)

[[tensor([149, 149, 149]), torch.Size([534, 6])]] [torch.Size([534, 6])]


In [185]:
for i, size in enumerate(last_basic[0][0]):
    print(i, size)

0 tensor(149)
1 tensor(149)
2 tensor(149)


In [186]:
out.size() 

torch.Size([534, 7])

In [187]:
for a, b in red.named_modules():
    print(a, b)

 UNet(
  (inp): InputLayer()
  (convBN): ConvBNBlock(
    (conv): SubmanifoldConvolution 1->6 C3
    (bnr): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
  )
  (output): OutputLayer()
  (linear): Linear(in_features=6, out_features=7, bias=True)
  (downsample): ModuleList(
    (0): ResidualBlock_downsample(
      (bnr1): BatchNormReLU(6,eps=0.0001,momentum=0.99,affine=True)
      (conv1): Convolution 6->12 C9/2
      (bnr2): BatchNormReLU(12,eps=0.0001,momentum=0.99,affine=True)
      (subconv): SubmanifoldConvolution 12->12 C9
      (conv2): Convolution 6->12 C9/2
      (add): AddTable()
    )
    (1): ResidualBlock_downsample(
      (bnr1): BatchNormReLU(12,eps=0.0001,momentum=0.99,affine=True)
      (conv1): Convolution 12->24 C9/2
      (bnr2): BatchNormReLU(24,eps=0.0001,momentum=0.99,affine=True)
      (subconv): SubmanifoldConvolution 24->24 C9
      (conv2): Convolution 12->24 C9/2
      (add): AddTable()
    )
    (2): ResidualBlock_downsample(
      (bnr1): BatchNormRe

In [188]:
del(red)

In [189]:
out

tensor([[ 0.0824,  0.1937, -0.3515,  ..., -0.9256, -1.2758, -0.9520],
        [-0.3366,  0.0195, -0.6197,  ..., -0.9725, -1.1526, -0.7588],
        [-0.3130, -0.1576, -0.5520,  ..., -0.6571, -0.5902, -0.0584],
        ...,
        [-0.0771, -0.0663,  0.4380,  ..., -0.1989,  0.0227, -0.5389],
        [-0.0139,  0.1377, -0.1028,  ..., -0.9613, -1.0926, -1.3951],
        [ 0.5572, -0.0855,  0.3013,  ..., -0.0433,  0.1639,  0.0118]],
       grad_fn=<AddmmBackward>)

In [190]:
for i in out:
    print(i.argmax() + 1)

tensor(4)
tensor(4)
tensor(4)
tensor(1)
tensor(4)
tensor(2)
tensor(4)
tensor(7)
tensor(4)
tensor(4)
tensor(2)
tensor(2)
tensor(4)
tensor(2)
tensor(6)
tensor(4)
tensor(3)
tensor(4)
tensor(4)
tensor(2)
tensor(3)
tensor(3)
tensor(2)
tensor(4)
tensor(2)
tensor(6)
tensor(4)
tensor(6)
tensor(1)
tensor(4)
tensor(4)
tensor(4)
tensor(2)
tensor(4)
tensor(2)
tensor(4)
tensor(3)
tensor(3)
tensor(4)
tensor(2)
tensor(3)
tensor(3)
tensor(4)
tensor(3)
tensor(2)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(3)
tensor(2)
tensor(3)
tensor(4)
tensor(4)
tensor(2)
tensor(1)
tensor(6)
tensor(3)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(1)
tensor(3)
tensor(2)
tensor(1)
tensor(1)
tensor(3)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(3)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(2)
tensor(7)
tensor(4)
tensor(4)
tensor(2)
tensor(4)
tensor(4)
tensor(4)
tensor(2)
tensor(2)
tensor(3)
tensor(3)
tensor(2)
tensor(4)
tensor(3)
tensor(4)
tensor(4)
tensor(4)
tensor(4)


In [191]:
lab

tensor([5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 2, 2, 5, 5, 5, 5, 2, 2, 5, 5, 5, 5, 5,
        5, 5, 5, 2, 2, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 2, 2, 2, 5, 5, 2, 2, 5,
        5, 5, 5, 2, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5,
        5, 5, 5, 2, 2, 5, 5, 2, 2, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5,
        2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2,
        2, 2, 5, 2, 2, 5, 5, 5, 5, 5, 5, 4, 6, 2, 1, 1, 3, 2, 1, 2, 2, 2, 5, 4,
        5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 4, 2, 2, 5, 4, 5, 3, 3, 5, 4, 6,
        3, 6, 6, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4,
        1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 2, 2, 5, 2, 5,
        5, 2, 5, 2, 5, 2, 2, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 2, 5,
        5, 2, 5, 5, 5, 5, 5, 2, 5, 2, 2, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 2,
        5, 6, 5, 2, 2, 5, 6, 6, 6, 6, 5, 5, 2, 5, 6, 6, 2, 5, 5, 2, 2, 5, 5, 2,
        2, 5, 2, 2, 5, 5, 5, 5, 5, 5, 5,

In [ ]:
#check if the job already exists                                                                                                 \
                                                                                                                                                                  
job = jobsdir + city + "_cut{cutnum}_{num}.job".format(cutnum = cutnum, num = num)
if os.path.exists(job):
    continue